# Introduction à la programmation Python

Julien Velcin, Université Lyon 2

Formation doctorale de l'UdL (2022-2023)

## TP n°3 : de la chaîne de caractères au sac de mots

## Differentes manières de représenter des données textuelles

- Chaîne de caractères (string)
- Bag-of-Words (BoW)
- Vector Space Model (VSM)
- Ajouter des méta-données (par ex. catégories grammaticales)
- Représentations plus complexes : arbres syntaxiques, graphes, etc.

Chaque représentation implique une manière différente de *comparer* les documents.

## Un exemple très simple

*"John Doe has bought an apple."*

Point de vue linguistique:

<table style="border:0;">
<tr style="border:0;">
<td style="border:0; white-space:pre; padding:0 100px 0 0px;">"John Doe has bought an apple."</td>
<td style="border:0;"><img src="img/syntaxtree.png" style='height: 150px'/></td>
</tr>
</table>

Point de vue statistique :

<img src="img/bow-illu.png" style='height: 150px'/>

<table style="border:0;">
<tr style="border:0;">
<td style="border:0; white-space:pre; padding:0 100px 0 0px;">"John Doe has bought an apple."</td>
<td style="border:0;">{ apple,<br/><br/> bought,<br/><br/>John_Doe } </td>
</tr>
</table>

<img src="img/bow.png" style='height: 400px'/>

On peut déjà observer des problèmes évidents, tel que :

*"Mary asked Fred out."*
<br/>
*"Fred asked Mary out."*

La représentation est la même :
<img src="img/mary.png" style='height: 200px'/>

A partir d'un *ensemble* de documents, on souhaite obtenir l'objet suivant :
<img src="img/termdocmatrix.png" style='height: 300px'/>

Par exemple :
<img src="img/termdocmatrix-2.jpg" style='width: 400px'/>

# Extraire les caractéristiques des textes

En pratique, on utilise le formalisme du "sac de mots" et on suit plusieurs étapes :

<img src="img/bag_of_words.svg" width="100%">


Prenons à présent deux documents, tels que :

In [2]:
X = ["Some say the world will end in fire,",
     "Some say in ice."]

In [3]:
len(X)

2

La librairie contient une fonction qui permet de "vectoriser" un ensemble de textes en prenant en compte un certain nombre de prétraitements (*preprocessing*) couramment employés : mis en minuscule, utilisation de mots-outils, etc.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X)

CountVectorizer()

Observons le vocabulaire automatiquement construit à partir de ces deux textes :

In [5]:
vectorizer.vocabulary_

{'some': 5,
 'say': 4,
 'the': 6,
 'world': 8,
 'will': 7,
 'end': 0,
 'in': 3,
 'fire': 1,
 'ice': 2}

Quelques mots sur la présence de mots tels que "in" ou "the"

Mots les plus fréquents observés dans Harry Potter :

<img src="img/zipf_1.png" width="100%">

Cette forme particulière est appelée "loi de Zipf". Elle a été observée pour la première fois par le linguiste George K. Zipf (1902-1950).

La loi de Zipf indique que si l'on observe un corpus suffisamment grand, la fréquence d'apparition d'un mot est **inversement proportionnelle** à son rang dans la table des fréquences.

Même graphique mais en supprimant les mots-outils anglais :
    
<img src="img/zipf_2.png" width="100%">

D'autres prétraitements sont également disponibles, tels que :

* racinisation (stemming) : trouver la racine des mots, comme dans :
     
> learn: learns, learned, learning...<br/>
> march: marcher, marchera, marcherai...
 
* lemmatisation (lemmatization) : trouver le lemme, comme dans :

> to be: am, are<br/>
> être : suis, sont...  

Le stemming nous aide à :

- rapprocher deux documents sémantiquement liés mais n'utilisant pas exactement les mêmes termes
- réduire la taille du vocabulaire (cf. malédiction de la dimension)

Ok pour :

    adventur: [adventure,adventurer,adventurers,adventures,adventurous]

mais...

    anim: [animal, animals, animation]



Autre prétraitement utile : remplacer certains motifs trouvés dans les textes

In [5]:
print(X)
str = [x.replace("world", "worlds") for x in X]
print(str)

['Some say the world will end in fire,', 'Some say in ice.']
['Some say the worlds will end in fire,', 'Some say in ice.']


In [6]:
import re
re.sub("[s|S]+\w*", "truc", X[0])

'truc truc the world will end in fire,'

Corriger automatiquement l'orthographe des mots grâce à la distance d'édition (ou distance Levenshtein, proposée en 1965).
Il s'agit de calculer le plus petit nombre d'opérations (insertion, suppression, remplacement) pour passer d'une chaîne à une autre.

In [7]:
# one possible implementation from https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [8]:
#levenshtein("chaîne", "chaine")
#levenshtein("chaîne", "chiane")
levenshtein("remerciement", "remerciments")

2

# Matrice documents x termes

On peut maintenant construire la matrice documents * termes :

In [6]:
X_bag_of_words = vectorizer.transform(X)

In [8]:
X_bag_of_words.shape

(2, 9)

In [9]:
X_bag_of_words

<2x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

Jetons un oeil au contenu de la matrice :

In [12]:
X_bag_of_words.toarray()

array([[1, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0]])

On observe que la valeur indiquée dans une cellule est le nombre d'occurrences d'un terme dans un document (TF pour *Term Frequency*)

On peut également retrouver le nom des termes du vocabulaire :

In [10]:
vectorizer.get_feature_names()

['end', 'fire', 'ice', 'in', 'say', 'some', 'the', 'will', 'world']

On peut retrouver les attributs (features) utilisés dans les documents :

In [14]:
vectorizer.inverse_transform(X_bag_of_words)

[array(['end', 'fire', 'in', 'say', 'some', 'the', 'will', 'world'],
       dtype='<U5'),
 array(['ice', 'in', 'say', 'some'], dtype='<U5')]

# Représentation TFxIDF
A la place du nombre d'occurrences (TF), on peut utiliser une autre mesure qui prend en compte la rareté d'un mot dans le corpus :

$$ tf_{t,d} \times idf_{t} $$

avec $tf_{t,d}$ le nombre d'occurrences de $t$ dans $d$

et $idf_{t} = \log \frac{N}{df_t}$ ($N$ est le nombre total de documents)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X)

TfidfVectorizer()

In [16]:
import numpy as np
np.set_printoptions(precision=2)

X_TFxIDF = tfidf_vectorizer.transform(X)
print(X_TFxIDF.toarray())

[[0.39 0.39 0.   0.28 0.28 0.28 0.39 0.39 0.39]
 [0.   0.   0.63 0.45 0.45 0.45 0.   0.   0.  ]]


# Comparaison de deux textes

Les distances usuelles (ex. euclidenne) ne sont pas adaptées.

Dans les espaces à **beaucoup de dimensions** :

Pourquoi les banquiers n'ont jamais de lingots sphériques ?

Pourquoi les marchands d'oranges occupent beaucoup de place pour empiler peu d'oranges ?

http://www.brouty.fr/Maths/sphere.html (see "Curiosités du calcul")

Malédiction de la dimension (curse of dimensionality)

Richard E. Bellman (1920-1984): les hypervolumes sont presque vides!

<img src="img/curse.png" style='height: 400px'/>

Un volume avec $dim=d$ a besoin de $10^d$ données pour peupler équitablement l'espace.

# Produit scalaire et cosinus

$\vec{x}$ et $\vec{y}$ sont deux vecteurs dans le VSM.

Cosine est une mesure de **similarité** calculée sur l'angle formé par les deux vecteurs :

$$cos(\vec{x},\vec{y}) = \frac{\vec{x}.\vec{y}}{||\vec{x}||_2 \times ||\vec{y}||_2}$$

Elle prend une valeur entre 0 (rien en commun) et 1 (même vecteurs à une constante près).

# Interprétation géométrique

<img src="img/cosine.png" style='height: 300px'/>

# Exemple : distribution des mots les plus fréquents dans Harry Potter

Commençons par lire les données :

In [17]:
import os

with open(os.path.join("datasets", "Harry_Potter_1.txt")) as f:
    lines = [line.strip() for line in f.readlines()]

tf_vectorizer = CountVectorizer()   
#tfidf_vectorizer = TfidfVectorizer()
tf_vectorizer.fit(lines)

# montrer l'intégralité du vocabulaire (peut être long) :
#tf_vectorizer.vocabulary_

CountVectorizer()

In [18]:
X_hp = tf_vectorizer.transform(lines)
features_hp = tf_vectorizer.get_feature_names()

On peut utiliser quelques fonctions pour afficher le vecteur pour un document en particulier :

In [19]:
import pandas as pd

# des options permettent de limiter (ou non) le nombre de lignes/colonnes affichées
# par exemple :
# pd.set_option('display.max_rows', None)

def top_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids if row[i]>0]
    df = pd.DataFrame(top_feats)
    if len(top_feats) > 0:
        df.columns = ['feature', 'score']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_feats(row, features, top_n)


In [20]:
print(lines[4])

top_feats_in_doc(X_hp, features_hp, 4, top_n=10)
#top_feats_in_doc(X_hp, features_hp, 6, top_n=50)

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.


,feature,score
0,they,3
1,were,3
2,to,2
3,you,2
4,thank,1
5,strange,1
6,dursley,1
7,mr,1
8,and,1
9,because,1


Afficher les mots les plus fréquents :

In [21]:
D = X_hp.toarray()

n_docs, n_terms = D.shape

#tf_means = np.mean(D, axis=0)
tf_sum = np.sum(D, axis=0)
top_feats(tf_sum, features_hp)

,feature,score
0,the,3627
1,and,1923
2,to,1861
3,he,1759
4,harry,1326
5,of,1267
6,it,1186
7,was,1186
8,you,1037
9,in,967


Comparer des documents revient à comparer les valeurs de deux colonnes (vecteurs).

In [22]:
import math

# fonction calculant le cosinus entre deux vecteurs
def cosinus(i, j):
    num = np.dot(i, j)
    den = math.sqrt(sum(i*i))*math.sqrt(sum(j*j))
    if (den>0):    
        return (num/den)
    else:
        return 0

In [23]:
print(top_feats_in_doc(X_hp, features_hp, 5, top_n=40))
print(top_feats_in_doc(X_hp, features_hp, 10, top_n=40))

      feature  score
0         the      4
1         was      4
2         and      3
3          of      3
4          he      2
5        very      2
6     dursley      2
7          in      2
8       which      2
9      called      2
10        had      2
11       neck      2
12       thin      1
13         mr      1
14     garden      1
15        man      1
16      their      1
17      beefy      1
18        big      1
19   director      1
20   anywhere      1
21       have      1
22      there      1
23       came      1
24      twice      1
25       made      1
26       much      1
27        mrs      1
28       firm      1
29         on      1
30  neighbors      1
31        did      1
32        she      1
33      small      1
34      spent      1
35  grunnings      1
36       over      1
37      usual      1
38         as      1
39    craning      1
     feature  score
0         he      2
1        got      1
2       tyke      1
3         mr      1
4       four      1
5       left      1

In [24]:
cosinus(D[5, :], D[10, :])

0.31859654643215496

# Créer son propre moteur de recherche

In [25]:
features_hp.index("wizard")

5600

In [26]:
#query = ['privet', 'drive', 'dursley']
#query = ['1473', 'aaaaarrrgh']
query = ['chess', 'chess', 'chess', 'play', 'harry']

indexes = [features_hp.index(q) for q in query if q in features_hp]
print(indexes)

[818, 818, 818, 3543, 2246]


On construit un vecteur de la même taille que le vocabulaire. Il est initialisé à zéro, puis on y met la valeur 1 pour les termes de la requête.

In [27]:
query_vec = np.zeros(n_terms)
query_vec[indexes] = 1

In [28]:
#len(D[10,:])
len(query_vec)
#query_vec[0:10]
#query_vec[3658:3670]

5707

On peut vérifier que le vecteur requête contient bien 3 éléments non nuls.

In [29]:
sum(query_vec)

3.0

Calcul du cosinus vis-à-vis de la requête pour 1 document :

In [30]:
cosinus(D[4, :], query_vec)
#lines[4]

0.0

On automatise en calculant pour tous les docs et on triant le résultat :

In [31]:
# fonction qui crée un dictionnaire associant le cosinus à chaque document
# puis le trie de manière décroissante

def search(q, D):
    cc = {i: cosinus(D[i, :], q) for i in range(n_docs)}
    cc = sorted(cc.items(), key=lambda x: x[1], reverse=True)
    return cc

In [32]:
result = search(query_vec, D)

In [33]:
result[0:10]

[(2137, 0.6546536707079772),
 (3135, 0.5773502691896258),
 (1060, 0.47140452079103173),
 (129, 0.40824829046386296),
 (815, 0.40824829046386296),
 (942, 0.40824829046386296),
 (970, 0.40824829046386296),
 (1073, 0.40824829046386296),
 (1224, 0.40824829046386296),
 (1352, 0.40824829046386296)]

On ne retient que les dix premiers résultats (par exemple).

In [34]:
nb_top_docs = 10
top_docs = [r for (r,v) in result[0:nb_top_docs]]
print(top_docs)

[2137, 3135, 1060, 129, 815, 942, 970, 1073, 1224, 1352]


Pour finir, on peut afficher les textes les plus pertinents pour la requête :

In [35]:
for i, td in zip(range(nb_top_docs), top_docs):
    #print(top_feats_in_doc(X_hp, features_hp, td))
    print("%s (%s): %s" % (i+1, td, lines[td]))

1 (2137): “Want to play chess, Harry?” said Ron.
2 (3135): “Harry!”
3 (1060): “Harry Potter,” said Harry.
4 (129): Harry groaned.
5 (815): Harry swallowed.
6 (942): “Harry Potter!”
7 (970): Harry nodded.
8 (1073): Harry stared.
9 (1224): “Potter, Harry!”
10 (1352): Dear Harry,


## Exercice pour les plus courageux

- choisir un corpus assez long (entre 500 et 3000 lignes)
- découpez le corpus en textes courts (par ex. des phrases ou des paragraphes)
- prétraitez-le de différentes manières :
    * TF et TFxIDF
    * taille du vocabulaire (par ex. 500 mots les plus fréquents vs. 2000 mots)
    * avec/sans les mots-outils
- déployer un moteur de recherche sur votre corpus
